# TERADATA NOS 
* Teradata Vantage’s NOS technology allows users to leverage the analytical power of Teradata Vantage against data in object stores such as Amazon’s S3 or Azure Blob Storage. NOS works by allowing Vantage users to simply set up a foreign table definition that maps to data that exists within an object store bucket.
* This allows users to query JSON, CSV, and Parquet format data within object stores just like local data within Teradata Vantage. The query process is dynamic and in real time, allowing the data to be analyzed without the need for it to be loaded and continually persisted in local Teradata Vantage storage

In [4]:
%connect tdprod1_chasing

Password: ··········


Success: 'tdprod1_chasing' connection established and activated for user 'chasing', with default database 'chasing'


# An authorization object is used to control who can access an external object store(Google Cloud Storage/Amazon S3/Azure Blob storage)
**<div class="alert alert-block alert-info"> Below is the syntax to create Authorization**
</div>

In [ ]:
CREATE AUTHORIZATION chasing.td_auth_sb_tdprod_files_nos
AS DEFINER TRUSTED
USER 'GOOGWNXKS7PSBZTJRSZGP3T6'
PASSWORD '********';

# Function mapping allows you to set a default for particular configuration settings without the user needing to provide them
**<div class="alert alert-block alert-info"> Below is the syntax to create Authorization**
</div>


In [6]:
drop function mapping chasing.READ_NOS_FM;

Success: 14 rows affected

In [10]:
CREATE FUNCTION MAPPING chasing.READ_NOS_FM
FOR READ_NOS EXTERNAL SECURITY DEFINER TRUSTED td_auth_sb_tdprod_files_nos
USING
BUFFERSIZE,
SAMPLE_PERC,
ROWFORMAT,
RETURNTYPE,
HEADER,
MANIFEST,
LOCATION,
STOREDAS,
FULLSCAN,
ANY IN TABLE;


ERROR: Unable to run SQL: Unable to run SQL query: Database reported error:5588:Function 'READ_NOS_FM' already exists

# READ_NOS (Allows you to read data stored in external storage)
- Perform an ad hoc query on CSV and JSON formatted data with the data in-place on an external object store
- List all the objects and path structure of an object store and read the schema of Parquet objects
- Bypass creating a foreign table in the Advanced SQL Engine

## Sampling External Data for CSV format Using READ_NOS
**<div class="alert alert-block alert-info"> Below is the syntax to see sample rows without column names, employee.csv file is placed in Google Cloud Storage**
</div>

In [11]:
SELECT TOP 2 payload.* FROM chasing.READ_NOS_FM (
ON ( SELECT CAST( NULL AS DATASET STORAGE FORMAT CSV ) )
USING
LOCATION('/gs/storage.googleapis.com/sb-tdprod-files-nos/gwl_data/training/employees.csv')
) AS D;

WARNING: [Teradata Database] [Warning 9986] 0 file(s) and 1 record(s) in foreign table(s) were skipped.

,PAYLOAD
1,"[""199"",""Douglas"",""Grant"",""DGRANT"",""650.507.9844"",""13-Jan-08""]"
2,"[""198"",""Donald"",""OConnell"",""DOCONNEL"",""650.507.9833"",""21-Jun-07""]"


## Sampling External Data for JSON format Using READ_NOS 
**<div class="alert alert-block alert-info"> Below is the syntax to see sample rows without column names, Language.json file is placed in Google Cloud Storage**
</div>

In [ ]:
SELECT TOP 2 payload..* FROM chasing.READ_NOS_FM (
ON ( SELECT CAST( NULL AS JSON ) )
USING
LOCATION('/gs/storage.googleapis.com/sb-tdprod-files-nos/gwl_data/training/Language.json')
) AS D;

## To Sample External Data for PARQUET format Using READ_NOS use Foriegn Table 
**<div class="alert alert-block alert-info"> Below is the syntax to get schema for PARQUET file which is placed in Google Cloud Storage**
</div>

In [ ]:
SELECT * FROM chasing.READ_NOS_FM (
USING
STOREDAS('PARQUET')
FULLSCAN('TRUE')
RETURNTYPE('NOSREAD_PARQUET_SCHEMA')
LOCATION ('/gs/storage.googleapis.com/sb-tdprod-files-nos/gwl_data/training/WRITE_NOS/')
) AS D;

**<div class="alert alert-block alert-info"> Use Schema from Above Query and Create Foriegn table for files present under Google Cloud Storage**
</div>

In [ ]:
CREATE FOREIGN TABLE CHASING.FT_SITE_PARQUET
, EXTERNAL SECURITY DEFINER TRUSTED TD_AUTH_SB_TDPROD_FILES_NOS
(
  LOCATION VARCHAR(2048) CHARACTER SET UNICODE CASESPECIFIC
  , SITE_NUM VARCHAR(16) CHARACTER SET UNICODE CASESPECIFIC
  , RGN_NUM VARCHAR(16) CHARACTER SET UNICODE CASESPECIFIC
  , EFF_DT DATE
)
USING (
    LOCATION ('/gs/storage.googleapis.com/sb-tdprod-files-nos/gwl_data/training/WRITE_NOS/')
    STOREDAS ('PARQUET')
) NO PRIMARY INDEX
, PARTITION BY COLUMN ;

In [ ]:
SELECT * FROM  CHASING.FT_SITE_PARQUET;

# WRITE_NOS 
- WRITE_NOS allows you to extract selected or all columns from a database table or from derived results and write to external object storage, such as Amazon S3/Google Cloud Storage.
- WRITE_NOS stores data in Parquet format.

**<div class="alert alert-block alert-info"> Create Authorization for WRITE_NOS**
</div>

In [ ]:
CREATE FUNCTION MAPPING chasing.WRITE_NOS_FM
FOR WRITE_NOS EXTERNAL SECURITY DEFINER TRUSTED td_auth_sb_tdprod_files_nos
USING
LOCATION, 
STOREDAS,
MAXOBJECTSIZE,
COMPRESSION,
NAMING,
INCLUDE_ORDERING,
INCLUDE_HASHBY,
MANIFESTFILE,
MANIFESTONLY,
OVERWRITE,
ANY IN TABLE;

## Writing to External Storage in PARQUET format Using WRITE_NOS
**<div class="alert alert-block alert-info"> Below is the syntax to write rows from a table to Google Cloud Storage in PARQUET format**
</div>

In [ ]:
SELECT NodeId, AmpId, Sequence, ObjectName, ObjectSize, RecordCount
FROM chasing.WRITE_NOS_FM (
     ON  ( SELECT SITE_NUM,RGN_NUM,EFF_DT FROM RLDMPROD_V.SITE_HIER WHERE EFF_DT >='2022-01-01')
        PARTITION BY EFF_DT ORDER BY EFF_DT
     USING
        LOCATION('/gs/storage.googleapis.com/sb-tdprod-files-nos/gwl_data/training/WRITE_NOS/')
        STOREDAS('PARQUET')
        COMPRESSION('GZIP')
        MAXOBJECTSIZE('4MB')
) AS d 
ORDER BY AmpId;

## Insert data Teradata Permanent table from Foriegn table
**<div class="alert alert-block alert-info">Create Foriegn table for CSV file placed at Google Cloud Storage using READ_NOS**
</div>

In [ ]:
CREATE FOREIGN TABLE CHASING.FT_EMPLOYEE_CSV
, EXTERNAL SECURITY DEFINER TRUSTED TD_AUTH_SB_TDPROD_FILES_NOS
(  Location VARCHAR(2048) CHARACTER SET UNICODE CASESPECIFIC,
   Payload DATASET INLINE LENGTH 64000 STORAGE FORMAT CSV
)
USING (LOCATION('/gs/storage.googleapis.com/sb-tdprod-files-nos/gwl_data/training/employees.csv'));



**<div class="alert alert-block alert-info"> Create Permanent Table and Insert table from Foriegn Table**
</div>

In [ ]:

CREATE TABLE  CHASING.PERM_EMPLOYEE_CSV
(EMPLOYEE_ID VARCHAR(20)
,FIRST_NAME VARCHAR(50)
,LAST_NAME VARCHAR(50)
,EMAIL VARCHAR(50)
,PHONE_NUMBER VARCHAR(50)
,HIRE_DATE VARCHAR(50)) 
PRIMARY INDEX (EMPLOYEE_ID);

In [ ]:
INSERT INTO CHASING.PERM_EMPLOYEE_CSV
SELECT
payload..EMPLOYEE_ID
,payload..FIRST_NAME
,payload..LAST_NAME
,payload..EMAIL
,payload..PHONE_NUMBER 
,payload..HIRE_DATE 
FROM CHASING.FT_EMPLOYEE_CSV;

In [ ]:
SELECT * FROM CHASING.PERM_EMPLOYEE_CSV;

In [23]:
SELECT TheDate AS LogDate
  ,cast(SUM(HostReadKB)*1e3/(1024*1024*1024) as bigint)  as Inbound_Bytes
  --,cast(SUM(HostWriteKB)*1e3 as bigint) as Outbound_Bytes
  FROM  PDCRInfo.ResUsageSpma_Hst -- ResUsageSPMA
  WHERE TheDate BETWEEN '2020-01-01' and current_date-1
  GROUP BY LogDate


,LogDate,Inbound_Bytes
1,2020-08-31,3942
2,2021-07-28,3185
3,2021-01-25,5043
4,2021-03-25,3133
5,2021-05-24,2969
6,2020-12-08,5436
7,2020-10-31,4282
8,2020-02-02,2356
9,2020-07-18,4792
10,2020-12-18,4455


In [1]:
%chart x=LOgDate,Y=Inbound_Bytes, Outboud_Bytes, Title="Teradata - Outbound and Inboud Data Transfer", labelx="Date", Labely="Inbout Bytes",mark="line"


ERROR: x column name 'LOgDate' is not a column in this result set 